In [8]:
import numpy as np
from scripts.data_utils import load_data, get_random_split, augment_data, get_multiple_splits
from scripts.augmentation_models import augmentaion_noise_jittering

In [5]:
path_to_open, path_to_close = '../data/opened_ihb.npy', '../data/opened_ihb.npy'
X, y, groups = load_data(path_to_open, path_to_close)
X_train, X_test, y_train, y_test, train_groups, test_groups = get_random_split(X, y,groups, test_size=0.15)

In [10]:
X_aug, y_aug, groups_aug, is_real_data = augment_data(augmentaion_noise_jittering,
                                                                                   X,
                                                                                   y,
                                                                                   groups,
                                                                                   n_aug=1)

X_train, X_test, y_train, y_test, train_groups, test_groups = get_random_split(X_aug,
                                                                               y_aug,
                                                                               groups_aug,
                                                                               is_real_data=is_real_data,
                                                                               test_size=0.15)

In [11]:
random_seeds = [42, 123, 456]
splits_with_seeds = get_multiple_splits(X_aug, y_aug, groups_aug, is_real_data, test_size=0.2, random_seeds=random_seeds)